# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [455]:
import import_ipynb
import importlib
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 200
n_product = 7
n_consumer = 1000
# market_variance = 0.01 #amount of variation between markets 
# endogenous_noise = 0.01 #how noise to generate price 

#structural parameters 
true_alpha = 2 # utility decreases with price 
true_beta = 3 # utility inceases with characteristics
true_sigma = 0.01 #variance of sensitivity to characteristics

import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)

importlib.reload(f)

changed third


ModuleSpec(name='BLP_functions', loader=<import_ipynb.NotebookLoader object at 0x10801c1a0>, origin='BLP_functions.ipynb')

In [456]:
all_markets = []
core_char = [1.2, 0.5, -0.8, 0.3, 1.0, -0.6, 0.9]

for market in range(n_market):
    # prod_char = np.random.uniform(-3,3,n_product)
    # prod_char = (prod_char - np.mean(prod_char)) / np.std(prod_char)
    prod_char = core_char + np.random.normal(0, 0.2, n_product) #generate product characteristics
    prod_char = (prod_char - np.mean(prod_char)) / np.std(prod_char)
    # generate market shares and price variables endogenously 
    cost_shifter = np.random.normal(0,2,n_product) #generate cost shifters
    price = f.price_gen(prod_char, cost_shifter) #generate price from characteristics and cost shifters
    price = (price - np.mean(price)) / np.std(price) #normalize price
    inner_utility = f.utility_gen(np.array([true_alpha, true_beta]), true_sigma, prod_char, price, n_consumer) 
    np.clip(inner_utility, -700, 700)
    # print(inner_utility.shape)
    # print("inner_utility", inner_utility)
    inner_prob_choice = f.choice_prob(np.array(inner_utility))
    # print("prob_choice", inner_prob_choice)
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=1)/inner_prob_choice.shape[1]
    # print("share", outer_market_share)
    # print(inner_prob_choice.shape)
    # generate the instrument now: calculate and normalize them 
    instrument_cost = np.random.normal(0,2,n_product)
    instrument_cost = (instrument_cost - np.mean(instrument_cost)) / np.std(instrument_cost)
    instrument_price = sum(price) - price
    instrument_price = (instrument_price - np.mean(instrument_price)) / np.std(instrument_price)
    instrument_char = sum(prod_char) - prod_char
    instrument_char = (instrument_char - np.mean(instrument_char)) / np.std(instrument_char)

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'prod_char': prod_char, 
        'price': price,
        'share': outer_market_share, 
        'true_delta': true_beta * prod_char - true_alpha * price,
        'share_not_buy': 1 - sum(outer_market_share),
        'instrument_cost': instrument_cost,
        'instrument_char': instrument_char,
        'instrument_price': instrument_price
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])

# print('true_beta', true_beta,'true_alpha', true_alpha, 'true_sigma', true_sigma)
export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_market_data.csv', index=False)

# print(np.unique(export_dataset['market_id']).size) #number of markets
# print(np.sum(export_dataset['market_id'] == 0)) #number of product

[   market_id  product_id  prod_char    price   share  true_delta  \
0          0           0    1.25250 -0.92984 0.97651     5.61718   
1          0           1   -0.02585 -0.02711 0.00347    -0.02333   
2          0           2   -1.80822 -0.93691 0.00010    -3.55082   
3          0           3   -0.10824  0.10666 0.00207    -0.53804   
4          0           4    0.93893  0.78879 0.01226     1.23919   
5          0           5   -0.91088 -0.93691 0.00150    -0.85882   
6          0           6    0.66176  1.93532 0.00054    -1.88536   

   share_not_buy  instrument_cost  instrument_char  instrument_price  
0        0.00355         -1.30356         -1.25250           0.92984  
1        0.00355         -0.90540          0.02585           0.02711  
2        0.00355          1.35541          1.80822           0.93691  
3        0.00355          0.60329          0.10824          -0.10666  
4        0.00355         -1.13794         -0.93893          -0.78879  
5        0.00355          0.